# Práctica Métodos de Simulación - Parte 3

## Elena Rivas, Teresa Grau, Ignacio Casso

## Enunciado

**Enfriamiento simulado. El problema de las n reinas.**

El problema de las n-reinas consiste en colocar n reinas en un tablero de ajedrez de
n x n de tal manera que no sea posible que dos reinas se capturen entre si, es decir,
que no estén en la misma fila, ni en la misma columna ni en la misma diagonal. Se
dice que hay una colisión si hay dos reinas que se pueden capturar entre si.

Se trata pues de encontrar una configuración (elegir las n celdas donde colocar a las
reinas) que minimice el número total de colisiones. Obtener la solución óptima para
n=9.

Mostrar gráficamente la evolución del valor de la función fitness y del parámetro
temperatura considerados a lo largo de las iteraciones de la metaheurı́stica.

## Solución

Proponemos dos implementaciones distintas para el algoritmo de enfraimiento simulado, que se diferencian en el espacio de búsqueda y en la codificación de las soluciones. En la primera consideramos todo el espacio de búsqueda, es dicir, todas las posibles configuraciones del tablero con $n$ reinas en él. En la segunda, observamos que por necesidad debe haber una reina en cada fila y otra en cada columna, y codificamos las soluciones como permutaciones de $n$ elementos, reduciendo el espacio de soluciones de un cardinal de $(n^2)^n$ a uno de $n!$. Para cada una de ellas se exploran distintas elecciones de parámetros

## Primera Solución

### Codificación de las soluciones

Codificaremos las soluciones como listas de 9 posiciones, siendo cada posición una tupla de 2 elementos. Para evitar redundancia en la codificación, se impone que esa lista esté ordenada (lexicográficamente), y para evitar soluciones no factibles, se prohiben los elementos repetidos en una lista.

### Función objetivo

La función objetivo a minimizar será el número de colisiones entre reinas.

In [3]:
def num_collisions(positions):
    
    if positions==[]:
        return 0
    else:
        queen, positions2 = positions[0], positions[1:]
        num_colls = 0
        for queen2 in positions2:
            if collision(queen,queen2):
                num_colls += 1
        return colls + num_collisions(positions2)
    
def collision(queen1,quenn2):
    i1,j1 = queen1
    i2,j2 = queen2
    
    if i1==i2 or j1==j2 or abs(i1-i2)==abs(j1-j2):
        return True
    else:
        return False

### Entorno de un punto en el espacio de bísqueda

## Segunda Solución